In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/distilbertbaseuncasedfinetunedsst2english/distilbert-base-uncased-finetuned-sst-2-english/rust_model.ot
/kaggle/input/distilbertbaseuncasedfinetunedsst2english/distilbert-base-uncased-finetuned-sst-2-english/config.json
/kaggle/input/distilbertbaseuncasedfinetunedsst2english/distilbert-base-uncased-finetuned-sst-2-english/README.md
/kaggle/input/distilbertbaseuncasedfinetunedsst2english/distilbert-base-uncased-finetuned-sst-2-english/tf_model.h5
/kaggle/input/distilbertbaseuncasedfinetunedsst2english/distilbert-base-uncased-finetuned-sst-2-english/tokenizer_config.json
/kaggle/input/distilbertbaseuncasedfinetunedsst2english/distilbert-base-uncased-finetuned-sst-2-english/pytorch_model.bin
/kaggle/input/distilbertbaseuncasedfinetunedsst2english/distilbert-base-uncased-finetuned-sst-2-english/.gitattributes
/kaggle/input/distilbertbaseuncasedfinetunedsst2english/distilbert-base-uncased-finetuned-sst-2-english/vocab.txt
/kaggle/input/distilbertbaseuncasedfinetunedsst2english

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import torch

from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer

In [6]:
train = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [7]:
test  = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")
test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [8]:
sample_sub = pd.read_csv("/kaggle/input/nlp-getting-started/sample_submission.csv")
sample_sub.head()

,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,0


In [9]:
X = train["text"].values
y = train["target"].values

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [10]:
train_ds = Dataset.from_dict({"text": X_train, "label": y_train})
val_ds   = Dataset.from_dict({"text": X_val, "label": y_val})
test_ds  = Dataset.from_dict({"text": test["text"]})

In [11]:
local_path = "/kaggle/input/distilbertbaseuncasedfinetunedsst2english/distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(local_path, local_files_only=True)
model = AutoModelForSequenceClassification.from_pretrained(local_path, num_labels=2, local_files_only=True)

In [12]:
def tokenize(batch):
    return tokenizer(batch["text"], truncation=True, padding="max_length", max_length=128)

train_ds = train_ds.map(tokenize, batched=True)
val_ds   = val_ds.map(tokenize, batched=True)
test_ds  = test_ds.map(tokenize, batched=True)

train_ds.set_format("torch", columns=["input_ids", "attention_mask", "label"])
val_ds.set_format("torch", columns=["input_ids", "attention_mask", "label"])
test_ds.set_format("torch", columns=["input_ids", "attention_mask"])

Map:   0%|          | 0/6090 [00:00<?, ? examples/s]

Map:   0%|          | 0/1523 [00:00<?, ? examples/s]

Map:   0%|          | 0/3263 [00:00<?, ? examples/s]

In [13]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds)
    return {"f1": f1}

In [14]:
training_args = TrainingArguments(
    output_dir="out",
    num_train_epochs=2,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    weight_decay=0.01,
    seed=42,
    report_to="none"
)

In [15]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    processing_class=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

Step,Training Loss
500,0.463500


TrainOutput(global_step=762, training_loss=0.4152523251030389, metrics={'train_runtime': 3442.537, 'train_samples_per_second': 3.538, 'train_steps_per_second': 0.221, 'total_flos': 403363228907520.0, 'train_loss': 0.4152523251030389, 'epoch': 2.0})

In [16]:
preds = trainer.predict(test_ds)
test_labels = preds.predictions.argmax(-1)

sample_sub["target"] = test_labels
sample_sub.to_csv("submission.csv", index=False)
print("Submission saved")

Submission saved
